In [ ]:
from bs4 import BeautifulSoup
import requests
import re
import codecs
from random import randint
from requests.compat import urljoin
from lxml import etree
from urllib.request import urlopen, Request
import random
import time
from itertools import cycle

In [ ]:
BASE_URL = 'https://www.thivien.net'
THO_VN_HIEN_DAI = 'https://www.thivien.net/searchpoem.php?PoemType=13&ViewType=1&Country=2&Age[]=3&Page='
GOOGLE_CACHE = 'http://webcache.googleusercontent.com/search?q=cache:'
MAX_PAGE = 954
i = 1
headers = {
        'User-Agent': 'Mozilla/6.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36'
    }

user_agent_list = None
with open('whatismybrowser-user-agent-database.txt', 'r') as file:
    user_agent_list = file.readlines()
agent_pool = cycle(user_agent_list)
def get_header():    
    #Pick a random user agent
    user_agent = next(agent_pool).replace('\n', '').replace('\b', '')
    #Set the headers 
    headers = {'User-Agent': user_agent}
    return headers

In [ ]:
get_header()

{'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.155 Safari/537.36'}

In [ ]:
proxies = None
with open('proxy1.txt', 'r') as file:
    proxies = file.readlines()
proxies = [proxy.replace('\n', '') for proxy in proxies]
proxy_pool = cycle(proxies)

In [ ]:
def get_page_links(loai_tho, page_number):
    url = loai_tho + str(page_number)
    page = requests.get(url, headers=get_header())
    page.encoding = 'utf-8'
    soup = BeautifulSoup(page.text, 'html.parser')
    page_link = soup.select('#PageContent > div.clearfix > div.page-content-main > div > h4 > a[href]')
    page_link = [GOOGLE_CACHE + urljoin(BASE_URL, link['href']) for link in page_link]
    print(page_link)
    return page_link

In [ ]:
def get_tho_content(page_link):
    url = Request(page_link, headers=get_header())
    main_page = urlopen(url)
    htmlparser = etree.HTMLParser()
    tree = etree.parse(main_page, htmlparser)
    return tree.xpath('//*[@id="PageContent"]/div[1]/div[2]/div[1]/div[3]/p/text()')

In [ ]:
def remove_number_punctual(s):
    return re.sub("[0-9\.,\(\)\?\:\\'\!]", '', s)
def process_tho(text):
    tho = text.copy()
    page_tho = []
    tho = [remove_number_punctual(t).strip().lower() for t in tho]
    tho_six_eight = []
    for t in tho:
        if len(t.split()) == 6 or len(t.split()) == 8:
            tho_six_eight.append(t)
    return tho_six_eight

In [ ]:
def save_tho(tho):
    file = codecs.open("tho.txt", "a+", "utf-8")
    for bai_tho in tho:
        for line in bai_tho:
            file.write(f'{line}\n')
        file.write('\n')
    file.close()

In [ ]:
page_links = get_page_links(THO_VN_HIEN_DAI, 1)
page_links

[]


[]

In [ ]:
for i in range(11, MAX_PAGE + 1):
    try:
        page_links = get_page_links(THO_VN_HIEN_DAI, i)
        print(i)
        for page_link in page_links:
            tho = []
            print(page_link)
            page_content = get_tho_content(page_link)
            page_content = process_tho(page_content)
            if len(page_content)%2 == 0:
                tho.append(page_content)
            save_tho(tho)
        time.sleep(20)
    except:
        
        continue

[]
11
[]
12
